In [4]:
import pandas as pd
import zipfile

In [8]:
beer_reviews_src = 'resources/beerreviews.zip'
us_breweries_src = 'resources/us-breweries.zip'

In [9]:
with zipfile.ZipFile(beer_reviews_src) as z:
   with z.open("beer_reviews.csv") as f:
      beer_reviews_src_df = pd.read_csv(f)

beer_reviews_src_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [18]:
beer_reviews_src_df['brewery_name'].nunique()

5742

In [10]:
with zipfile.ZipFile(us_breweries_src) as z:
   with z.open("breweries_us.csv") as f:
      us_breweries_src_df = pd.read_csv(f)
us_breweries_src_df.head()

,brewery_name,type,address,website,state,state_breweries
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


In [19]:
us_breweries_src_df['brewery_name'].nunique()

2308

In [12]:
combined_beer = beer_reviews_src_df.merge(us_breweries_src_df,left_on='brewery_name', right_on='brewery_name', how='l')

In [17]:
combined_beer['brewery_name'].nunique()

351